<a href="https://colab.research.google.com/github/Jdean4747/Data5600HW/blob/main/HW5_5600.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/nyc-rolling-sales_clean.csv")

In [ ]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,sale_year,sale_month
0,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,10,0,10,2272.0,6794.0,1913,2,C4,3936272.0,2016-09-23,2016,9
1,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,8,0,8,1750.0,4226.0,1920,2,C4,3192840.0,2016-09-23,2016,9
2,Manhattan,ALPHABET CITY,14 RENTALS - 4-10 UNIT,2A,S3,10009,3,1,4,1520.0,3360.0,1910,2,S3,3300000.0,2016-11-15,2016,11
3,Manhattan,ALPHABET CITY,22 STORE BUILDINGS,4,K4,10009,0,1,1,1218.0,3586.0,1899,4,K4,3700000.0,2016-09-28,2016,9
4,Manhattan,CHELSEA,07 RENTALS - WALKUP APARTMENTS,2B,C4,10011,9,0,9,1566.0,6330.0,1901,2,C4,1583840.0,2017-08-31,2017,8


In [ ]:
df['BOROUGH'] = df['BOROUGH'].astype('category')
df['NEIGHBORHOOD'] = df['NEIGHBORHOOD'].astype('category')
df['BUILDING CLASS CATEGORY'] = df['BUILDING CLASS CATEGORY'].astype('category')
df['TAX CLASS AT PRESENT'] = df['TAX CLASS AT PRESENT'].astype('category')
df['BUILDING CLASS AT PRESENT'] = df['BUILDING CLASS AT PRESENT'].astype('category')
df['TAX CLASS AT TIME OF SALE'] = df['TAX CLASS AT TIME OF SALE'].astype('category')
df['BUILDING CLASS AT TIME OF SALE'] = df['BUILDING CLASS AT TIME OF SALE'].astype('category')

df['SALE PRICE'] = df['SALE PRICE'].astype('float64')
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].astype('float64')
df['YEAR BUILT'] = df['YEAR BUILT'].astype('float64')
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].astype('float64')

In [ ]:
df['target'] = np.log(df['SALE PRICE'])
df.drop(columns=['SALE PRICE'],inplace=True,axis=1)

In [ ]:
x = df[['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT', 'BUILDING CLASS AT PRESENT', 'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE']]
x = pd.get_dummies(x, drop_first=True)

In [ ]:
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_hat_test = model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_hat_test))
rmse_test

0.542704768668823

In [ ]:
resid = y_test - y_hat_test
log_predictions = pd.DataFrame({'y_test':y_test, 'y_hat_test':y_hat_test, 'resid':resid})
log_predictions.head()

,y_test,y_hat_test,resid
19745,13.779883,13.506745,0.273138
17399,13.800397,13.575325,0.225072
11376,14.731801,13.948092,0.783710
20957,12.100712,13.613374,-1.512662
2129,12.491252,13.021566,-0.530315


In [ ]:
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)))

rmse_cv5 = cross_val_score(model, x, y, cv=5, scoring=rmse_scorer)
print(rmse_cv5.mean())
rmse_cv10 = cross_val_score(model, x, y, cv=10, scoring=rmse_scorer)
print(rmse_cv10.mean())

0.6603832630820643
0.631346287839677


What I am seeing seems to be overfitting of the data. The RMSE score in the RMSE_test is much lower than the RMSE_CV scores. This could mean that I have too many variables as dummies which I do have a lot so it would make sense.